# 第 8 章 提升方法

## Boost
**“装袋”（bagging）和“提升”（boost）**是构建组合模型的两种最主要的方法，所谓的组合模型是由多个基本模型构成的模型，组合模型的预测效果往往比任意一个基本模型的效果都要好。
- 装袋：每个基本模型由从总体样本中随机抽样得到的不同数据集进行训练得到，通过重抽样得到不同训练数据集的过程称为装袋。
- 提升：每个基本模型训练时的数据集采用不同权重，针对上一个基本模型分类错误的样本增加权重，使得新的模型重点关注误分类样本
## AdaBoost
AdaBoost是AdaptiveBoost的缩写，表明该算法是具有适应性的提升算法。
1. 给每个训练样本（$x_{1},x_{2},….,x_{N}$）分配权重，初始权重$w_{1}$均为1/N。

2. 针对带有权值的样本进行训练，得到模型$G_m$（初始模型为G1）。

3. 计算模型$G_m$的误分率$e_m=\sum_{i=1}^Nw_iI(y_i\not= G_m(x_i))$

4. 计算模型$G_m$的系数$\alpha_m=0.5\log[(1-e_m)/e_m]$

5. 根据误分率e和当前权重向量$w_m$更新权重向量$w_{m+1}$。

6. 计算组合模型$f(x)=\sum_{m=1}^M\alpha_mG_m(x_i)$的误分率。

7.  当组合模型的误分率或迭代次数低于一定阈值，停止迭代；否则，回到步骤2）

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# processing data for model.
def create_data():
  iris = load_iris()
  df = pd.DataFrame(iris.data, columns=iris.feature_names)
  df['label'] = iris.target
  df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
  data = np.array(df.iloc[:100, [0,1,-1]])
  return data[:,:2], data[:,-1]

# loadding data
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("The shape of train data: X-{}, y-{}".format(X_train.shape, y_train.shape))
print("The shape of test data: X-{}, y-{}".format(X_test.shape, y_test.shape))

The shape of train data: X-(70, 2), y-(70,)
The shape of test data: X-(30, 2), y-(30,)


In [10]:
# the class of AdaBoost
class AdaBoost:
  def __init__(self, n_estimators=50, learning_rate=1.0):
      self.clf_num = n_estimators
      self.learning_rate = learning_rate

  def init_args(self, datasets, labels):
      self.X = datasets
      self.Y = labels
      self.M, self.N = datasets.shape
      # 弱分类器数目和集合
      self.clf_sets = []
      # 初始化weights
      self.weights = [1.0 / self.M] * self.M
      # G(x)系数 alpha
      self.alpha = []

  def _G(self, features, labels, weights):
      m = len(features)
      error = 100000.0  # 无穷大
      best_v = 0.0
      # 单维features
      features_min = min(features)
      features_max = max(features)
      n_step = (features_max - features_min + self.learning_rate) // self.learning_rate
      # print('n_step:{}'.format(n_step))
      direct, compare_array = None, None
      for i in range(1, int(n_step)):
          v = features_min + self.learning_rate * i
          if v not in features:
              # 误分类计算
              compare_array_positive = np.array([1 if features[k] > v else -1 for k in range(m)])
              weight_error_positive = sum([ weights[k] for k in range(m) if compare_array_positive[k] != labels[k] ])
              compare_array_nagetive = np.array([-1 if features[k] > v else 1 for k in range(m)])
              weight_error_nagetive = sum([ weights[k] for k in range(m) if compare_array_nagetive[k] != labels[k] ])
              if weight_error_positive < weight_error_nagetive:
                  weight_error = weight_error_positive
                  _compare_array = compare_array_positive
                  direct = 'positive'
              else:
                  weight_error = weight_error_nagetive
                  _compare_array = compare_array_nagetive
                  direct = 'nagetive'

              # print('v:{} error:{}'.format(v, weight_error))
              if weight_error < error:
                  error = weight_error
                  compare_array = _compare_array
                  best_v = v
      return best_v, direct, error, compare_array

  # 计算alpha
  def _alpha(self, error):
      return 0.5 * np.log((1 - error) / error)

  # 规范化因子
  def _Z(self, weights, a, clf):
      return sum([ weights[i] * np.exp(-1 * a * self.Y[i] * clf[i]) for i in range(self.M) ])

  # 权值更新
  def _w(self, a, clf, Z):
      for i in range(self.M):
          self.weights[i] = self.weights[i] * np.exp(-1 * a * self.Y[i] * clf[i]) / Z

  # G(x)的线性组合
  def _f(self, alpha, clf_sets):
      pass

  def G(self, x, v, direct):
      if direct == 'positive':
          return 1 if x > v else -1
      else:
          return -1 if x > v else 1

  def fit(self, X, y):
      self.init_args(X, y)
      for epoch in range(self.clf_num):
          best_clf_error, best_v, clf_result = 100000, None, None
          # 根据特征维度, 选择误差最小的
          for j in range(self.N):
              features = self.X[:, j]
              # 分类阈值，分类误差，分类结果
              v, direct, error, compare_array = self._G(features, self.Y, self.weights)
              if error < best_clf_error:
                  best_clf_error = error
                  best_v = v
                  final_direct = direct
                  clf_result = compare_array
                  axis = j
              # print('epoch:{}/{} feature:{} error:{} v:{}'.format(epoch, self.clf_num, j, error, best_v))
              if best_clf_error == 0:
                  break
          # 计算G(x)系数a
          a = self._alpha(best_clf_error)
          self.alpha.append(a)
          # 记录分类器
          self.clf_sets.append((axis, best_v, final_direct))
          # 规范化因子
          Z = self._Z(self.weights, a, clf_result)
          # 权值更新
          self._w(a, clf_result, Z)
          # print('classifier:{}/{} error:{:.3f} v:{} direct:{} a:{:.5f}'.format(epoch+1, self.clf_num, error, best_v, final_direct, a))
          # print('weight:{}'.format(self.weights))
          # print('\n')

  def predict(self, feature):
      result = 0.0
      for i in range(len(self.clf_sets)):
          axis, clf_v, direct = self.clf_sets[i]
          f_input = feature[axis]
          result += self.alpha[i] * self.G(f_input, clf_v, direct)
      # sign
      return 1 if result > 0 else -1

  def accuracy(self, X_test, y_test):
      num_correct = 0
      for i in range(len(X_test)):
          feature = X_test[i]
          if self.predict(feature) == y_test[i]:
              num_correct += 1

      return num_correct / len(X_test)
    
# -----------------------------------------
# TEST 例 8.1
X = np.arange(10).reshape(10, 1)
y = np.array([1, 1, 1, -1, -1, -1, 1, 1, 1, -1])

clf = AdaBoost(n_estimators=3, learning_rate=0.5)
clf.fit(X, y)
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
clf = AdaBoost(n_estimators=10, learning_rate=0.2)
clf.fit(X_train, y_train)
print("The accuracy of AdaBoost model: {}".format(clf.accuracy(X_test, y_test)))

The accuracy of AdaBoost model: 0.45454545454545453


## sklearn instance

In [12]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.5)
clf.fit(X_train, y_train)
print("The accuracy of AdaBoost model: {}".format(clf.score(X_test, y_test)))

The accuracy of AdaBoost model: 1.0


## 习题 8.1

In [13]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

# 加载训练数据
X = np.array([[0, 1, 3], [0, 3, 1], [1, 2, 2], [1, 1, 3], [1, 2, 3], [0, 1, 2],
              [1, 1, 2], [1, 1, 1], [1, 3, 1], [0, 2, 1]])
y = np.array([-1, -1, -1, -1, -1, -1, 1, 1, -1, -1])

clf = AdaBoostClassifier()
clf.fit(X, y)
y_predict = clf.predict(X)
score = clf.score(X, y)
print("原始输出:", y)
print("预测输出:", y_predict)
print("预测正确率：{:.2%}".format(score))

原始输出: [-1 -1 -1 -1 -1 -1  1  1 -1 -1]
预测输出: [-1 -1 -1 -1 -1 -1  1  1 -1 -1]
预测正确率：100.00%


In [14]:
# 习题 8.1
import numpy as np

# the class of AdaBoost
class AdaBoost:
  def __init__(self, X, y, tol=0.05, max_iter=10):
      # 训练数据 实例
      self.X = X
      # 训练数据 标签
      self.y = y
      # 训练中止条件 right_rate>self.tol
      self.tol = tol
      # 最大迭代次数
      self.max_iter = max_iter
      # 初始化样本权重w
      self.w = np.full((X.shape[0]), 1 / X.shape[0])
      self.G = []  # 弱分类器

  def build_stump(self):
      """
      以带权重的分类误差最小为目标，选择最佳分类阈值
      best_stump['dim'] 合适的特征所在维度
      best_stump['thresh']  合适特征的阈值
      best_stump['ineq']  树桩分类的标识lt,rt
      """
      m, n = np.shape(self.X)
      # 分类误差
      e_min = np.inf
      # 小于分类阈值的样本属于的标签类别
      sign = None
      # 最优分类树桩
      best_stump = {}
      for i in range(n):
          range_min = self.X[:, i].min()  # 求每一种特征的最大最小值
          range_max = self.X[:, i].max()
          step_size = (range_max - range_min) / n
          for j in range(-1, int(n) + 1):
              thresh_val = range_min + j * step_size
              # 计算左子树和右子树的误差
              for inequal in ['lt', 'rt']:
                  predict_vals = self.base_estimator(self.X, i, thresh_val, inequal)
                  err_arr = np.array(np.ones(m))
                  err_arr[predict_vals.T == self.y.T] = 0
                  weighted_error = np.dot(self.w, err_arr)
                  if weighted_error < e_min:
                      e_min = weighted_error
                      sign = predict_vals
                      best_stump['dim'] = i
                      best_stump['thresh'] = thresh_val
                      best_stump['ineq'] = inequal
      return best_stump, sign, e_min

  def updata_w(self, alpha, predict):
      """
      更新样本权重w
      """
      # 以下2行根据公式8.4 8.5 更新样本权重
      P = self.w * np.exp(-alpha * self.y * predict)
      self.w = P / P.sum()

  @staticmethod
  def base_estimator(X, dimen, threshVal, threshIneq):
      """
      计算单个弱分类器（决策树桩）预测输出
      """
      ret_array = np.ones(np.shape(X)[0])  # 预测矩阵
      # 左叶子 ，整个矩阵的样本进行比较赋值
      if threshIneq == 'lt':
          ret_array[X[:, dimen] <= threshVal] = -1.0
      else:
          ret_array[X[:, dimen] > threshVal] = -1.0
      return ret_array

  def fit(self):
      """
      对训练数据进行学习
      """
      G = 0
      for i in range(self.max_iter):
          best_stump, sign, error = self.build_stump()  # 获取当前迭代最佳分类阈值
          alpha = 1 / 2 * np.log((1 - error) / error)  # 计算本轮弱分类器的系数
          # 弱分类器权重
          best_stump['alpha'] = alpha
          # 保存弱分类器
          self.G.append(best_stump)
          # 以下3行计算当前总分类器（之前所有弱分类器加权和）分类效率
          G += alpha * sign
          y_predict = np.sign(G)
          error_rate = np.sum(np.abs(y_predict - self.y)) / 2 / self.y.shape[0]
          if error_rate < self.tol:  # 满足中止条件 则跳出循环
              print("迭代次数:", i + 1)
              break
          else:
              self.updata_w(alpha, y_predict)  # 若不满足，更新权重，继续迭代

  def predict(self, X):
      """
      对新数据进行预测
      """
      m = np.shape(X)[0]
      G = np.zeros(m)
      for i in range(len(self.G)):
          stump = self.G[i]
          # 遍历每一个弱分类器，进行加权
          _G = self.base_estimator(X, stump['dim'], stump['thresh'], stump['ineq'])
          alpha = stump['alpha']
          G += alpha * _G
      y_predict = np.sign(G)
      return y_predict.astype(int)

  def score(self, X, y):
      """对训练效果进行评价"""
      y_predict = self.predict(X)
      error_rate = np.sum(np.abs(y_predict - y)) / 2 / y.shape[0]
      return 1 - error_rate
    
# --------------------------------
# TEST
clf = AdaBoost(X, y)
clf.fit()
y_predict = clf.predict(X)
score = clf.score(X, y)
print("原始输出:", y)
print("预测输出:", y_predict)
print("预测正确率：{:.2%}".format(score))

迭代次数: 8
原始输出: [-1 -1 -1 -1 -1 -1  1  1 -1 -1]
预测输出: [-1 -1 -1 -1 -1 -1  1  1 -1 -1]
预测正确率：100.00%
